In [ ]:
import datetime
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
import warnings
import torch
import torch.nn as nn
import torch.nn.utils as torch_utils
import torch.optim as optim
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
import os
import random

from tqdm.auto import tqdm
from sklearn.preprocessing import LabelEncoder, MinMaxScaler
from google.colab import drive


plt.style.use('seaborn')
warnings.filterwarnings('ignore')
%matplotlib inline

drive.mount('/content/drive')
device = torch.device('cuda') if torch.cuda.is_available() else torch.device('cpu')

<ipython-input-1-534501904935>:21: MatplotlibDeprecationWarning: The seaborn styles shipped by Matplotlib are deprecated since 3.6, as they no longer correspond to the styles shipped by seaborn. However, they will remain available as 'seaborn-v0_8-<style>'. Alternatively, directly use the seaborn API instead.
  plt.style.use('seaborn')


Mounted at /content/drive


# Load Data

In [ ]:
brand = pd.read_csv('brand_keyword_cnt.csv')

sales = pd.read_csv('sales.csv')

train = pd.read_csv('train.csv')

# make price_new

In [ ]:
# 일 총금액 / 일 판매량 = 일 상품 하나당 가격

df = sales.iloc[:, 6:].div(train.iloc[:, 6:])

In [ ]:
df.head()

,2022-01-01,2022-01-02,2022-01-03,2022-01-04,2022-01-05,2022-01-06,2022-01-07,2022-01-08,2022-01-09,2022-01-10,...,2023-03-26,2023-03-27,2023-03-28,2023-03-29,2023-03-30,2023-03-31,2023-04-01,2023-04-02,2023-04-03,2023-04-04
0,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
1,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,22400.0,22400.0,22400.0,NaN,NaN,22400.0,NaN
2,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
3,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN
4,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN


In [ ]:
df = df.fillna(0) # 결측치 채우기

In [ ]:
df.head()

,2022-01-01,2022-01-02,2022-01-03,2022-01-04,2022-01-05,2022-01-06,2022-01-07,2022-01-08,2022-01-09,2022-01-10,...,2023-03-26,2023-03-27,2023-03-28,2023-03-29,2023-03-30,2023-03-31,2023-04-01,2023-04-02,2023-04-03,2023-04-04
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,22400.0,22400.0,22400.0,0.0,0.0,22400.0,0.0
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0


In [ ]:
# 원래 상품 가격은 일 가격 중 최대 값 (상품 가격이 오르지 않았다고 생각)

df['origin_price'] = df.iloc[:, :].max(axis=1)

In [ ]:
df.head()

,2022-01-01,2022-01-02,2022-01-03,2022-01-04,2022-01-05,2022-01-06,2022-01-07,2022-01-08,2022-01-09,2022-01-10,...,2023-03-27,2023-03-28,2023-03-29,2023-03-30,2023-03-31,2023-04-01,2023-04-02,2023-04-03,2023-04-04,origin_price
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,13500.000000
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,22400.0,22400.0,22400.0,0.0,0.0,22400.0,0.0,37800.000000
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,12900.000000
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,5583.333333
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,6480.000000


In [ ]:
## 스케일링 (원래 가격에서 일가격을 나누고 1을 빼면 몇프로 할인을 했는지 확인 가능 0 ~ 1 사이의 값) => 할인율을 구할 수 있음


# 마지막 열이 0이 아닌 행만 선택
mask = df.iloc[:, -1] != 0

# 해당 행의 2번째부터 마지막에서 두 번째 열까지의 위치를 지정
cols = df.columns[2:-1]

# 값을 계산하고 1인 값을 0으로 대체
df.loc[mask, cols] = (1 - (df.loc[mask, cols].div(df.loc[mask, df.columns[-1]], axis=0))).replace(1, 0)


In [ ]:
df = df.round(3) # 소수점 3자리까지 표현

In [ ]:
df.head()

,2022-01-01,2022-01-02,2022-01-03,2022-01-04,2022-01-05,2022-01-06,2022-01-07,2022-01-08,2022-01-09,2022-01-10,...,2023-03-27,2023-03-28,2023-03-29,2023-03-30,2023-03-31,2023-04-01,2023-04-02,2023-04-03,2023-04-04,origin_price
0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000,0.000,0.000,0.0,0.0,0.000,0.0,13500.000
1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.407,0.407,0.407,0.0,0.0,0.407,0.0,37800.000
2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000,0.000,0.000,0.0,0.0,0.000,0.0,12900.000
3,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000,0.000,0.000,0.0,0.0,0.000,0.0,5583.333
4,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.000,0.000,0.000,0.0,0.0,0.000,0.0,6480.000


In [ ]:
# 컬럼 순서 변경: origin_price를 맨 앞으로
cols = list(df.columns)

new_col = [cols[-1]] + cols[:-1]

df = df[new_col]

In [ ]:
df.to_csv('price_new.csv', index=False)

# train, brand df scale

In [ ]:
train.drop(['ID', '제품', '중분류', '소분류'], axis=1, inplace=True) # 필요없는 피쳐 제거

In [ ]:
# 숫자형 변수들의 min-max scaling을 수행하는 코드입니다.
numeric_cols = train.columns[2:]


### train 스케일링 ###

# train의 각 column의 min 및 max 계산
min_values_train = train[numeric_cols].min(axis=1)
max_values_train = train[numeric_cols].max(axis=1)

# 각 행의 범위(max-min)를 계산하고, 범위가 0인 경우 1로 대체
ranges_train = max_values_train - min_values_train
ranges_train[ranges_train == 0] = 1

# min-max scaling 수행
train[numeric_cols] = (train[numeric_cols].subtract(min_values_train, axis=0)).div(ranges_train, axis=0)

# max와 min 값을 dictionary 형태로 저장
scale_min_dict = min_values_train.to_dict()
scale_max_dict = max_values_train.to_dict()

In [ ]:
import pickle

with open('scale_min_dict.pkl', 'wb') as f:
    pickle.dump(scale_min_dict, f)

with open('scale_max_dict.pkl', 'wb') as f:
    pickle.dump(scale_max_dict, f)

In [ ]:
train.head(2)

,대분류,브랜드,2022-01-01,2022-01-02,2022-01-03,2022-01-04,2022-01-05,2022-01-06,2022-01-07,2022-01-08,...,2023-03-26,2023-03-27,2023-03-28,2023-03-29,2023-03-30,2023-03-31,2023-04-01,2023-04-02,2023-04-03,2023-04-04
0,B002-C001-0002,B002-00001,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000,0.000000,0.000000,0.0,0.0,0.000000,0.0
1,B002-C001-0003,B002-00002,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.111111,0.333333,0.222222,0.0,0.0,0.222222,0.0


In [ ]:
train = train.round(3)

In [ ]:
# 숫자형 변수들의 min-max scaling을 수행하는 코드입니다.
numeric_cols = brand.columns[1:]


### brand 스케일링 ###

# train의 각 column의 min 및 max 계산
min_values_brand = brand[numeric_cols].min(axis=1)
max_values_brand = brand[numeric_cols].max(axis=1)

# 각 행의 범위(max-min)를 계산하고, 범위가 0인 경우 1로 대체
ranges_brand = max_values_brand - min_values_brand
ranges_brand[ranges_brand == 0] = 1

# min-max scaling 수행
brand[numeric_cols] = (brand[numeric_cols].subtract(min_values_brand, axis=0)).div(ranges_brand, axis=0)


In [ ]:
brand.head()

,브랜드,2022-01-01,2022-01-02,2022-01-03,2022-01-04,2022-01-05,2022-01-06,2022-01-07,2022-01-08,2022-01-09,...,2023-03-26,2023-03-27,2023-03-28,2023-03-29,2023-03-30,2023-03-31,2023-04-01,2023-04-02,2023-04-03,2023-04-04
0,B002-00001,0.214576,0.234817,0.384615,0.655870,0.502025,0.421053,0.323886,0.307693,0.299596,...,0.068825,0.089069,0.085019,0.117410,0.000000,0.060729,0.068825,0.044535,0.072875,0.105263
1,B002-00002,0.230633,0.463479,0.312528,0.233732,0.258521,0.262948,0.208942,0.319610,0.274458,...,0.158034,0.209827,0.169544,0.162461,0.164675,0.134130,0.127933,0.162904,0.185038,0.192120
2,B002-00003,0.004359,0.006707,0.005030,0.000671,0.001677,0.007378,0.006372,0.003353,0.005365,...,0.009054,0.012743,0.007042,0.005701,0.020456,0.008048,0.017773,0.015091,0.020121,0.017103
3,B002-00005,0.094303,0.180747,0.222004,0.208251,0.218075,0.239686,0.176818,0.135560,0.141454,...,0.249510,0.288801,0.337918,0.271121,0.223969,0.157171,0.131631,0.119843,0.263261,0.261297
4,B002-00006,0.000000,0.000000,0.091544,0.119711,0.119711,0.119711,0.183100,0.042244,0.126755,...,0.035211,0.140844,0.042244,0.000000,0.063378,0.000000,0.000000,0.035211,0.105633,0.000000


In [ ]:
brand = brand.round(3)

In [ ]:
brand.head()

,브랜드,2022-01-01,2022-01-02,2022-01-03,2022-01-04,2022-01-05,2022-01-06,2022-01-07,2022-01-08,2022-01-09,...,2023-03-26,2023-03-27,2023-03-28,2023-03-29,2023-03-30,2023-03-31,2023-04-01,2023-04-02,2023-04-03,2023-04-04
0,B002-00001,0.215,0.235,0.385,0.656,0.502,0.421,0.324,0.308,0.300,...,0.069,0.089,0.085,0.117,0.000,0.061,0.069,0.045,0.073,0.105
1,B002-00002,0.231,0.463,0.313,0.234,0.259,0.263,0.209,0.320,0.274,...,0.158,0.210,0.170,0.162,0.165,0.134,0.128,0.163,0.185,0.192
2,B002-00003,0.004,0.007,0.005,0.001,0.002,0.007,0.006,0.003,0.005,...,0.009,0.013,0.007,0.006,0.020,0.008,0.018,0.015,0.020,0.017
3,B002-00005,0.094,0.181,0.222,0.208,0.218,0.240,0.177,0.136,0.141,...,0.250,0.289,0.338,0.271,0.224,0.157,0.132,0.120,0.263,0.261
4,B002-00006,0.000,0.000,0.092,0.120,0.120,0.120,0.183,0.042,0.127,...,0.035,0.141,0.042,0.000,0.063,0.000,0.000,0.035,0.106,0.000


# train, brand df 라벨링

In [ ]:

# Label Encoding
label_encoder = LabelEncoder()
categorical_columns = ['대분류', '브랜드']

for col in categorical_columns:
    label_encoder.fit(train[col])
    train[col] = label_encoder.transform(train[col])

    if col == '브랜드':
        brand[col] = label_encoder.transform(brand[col])

In [ ]:
train.head()

,대분류,브랜드,2022-01-01,2022-01-02,2022-01-03,2022-01-04,2022-01-05,2022-01-06,2022-01-07,2022-01-08,...,2023-03-26,2023-03-27,2023-03-28,2023-03-29,2023-03-30,2023-03-31,2023-04-01,2023-04-02,2023-04-03,2023-04-04
0,1,0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000,0.000,0.000,0.0,0.0,0.000,0.0
1,2,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.111,0.333,0.222,0.0,0.0,0.222,0.0
2,2,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000,0.000,0.000,0.0,0.0,0.000,0.0
3,2,1,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000,0.000,0.000,0.0,0.0,0.000,0.0
4,0,2,0.0,0.0,0.0,0.0,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000,0.000,0.000,0.0,0.0,0.000,0.0


In [ ]:
brand.head()

,브랜드,2022-01-01,2022-01-02,2022-01-03,2022-01-04,2022-01-05,2022-01-06,2022-01-07,2022-01-08,2022-01-09,...,2023-03-26,2023-03-27,2023-03-28,2023-03-29,2023-03-30,2023-03-31,2023-04-01,2023-04-02,2023-04-03,2023-04-04
0,0,0.215,0.235,0.385,0.656,0.502,0.421,0.324,0.308,0.300,...,0.069,0.089,0.085,0.117,0.000,0.061,0.069,0.045,0.073,0.105
1,1,0.231,0.463,0.313,0.234,0.259,0.263,0.209,0.320,0.274,...,0.158,0.210,0.170,0.162,0.165,0.134,0.128,0.163,0.185,0.192
2,2,0.004,0.007,0.005,0.001,0.002,0.007,0.006,0.003,0.005,...,0.009,0.013,0.007,0.006,0.020,0.008,0.018,0.015,0.020,0.017
3,3,0.094,0.181,0.222,0.208,0.218,0.240,0.177,0.136,0.141,...,0.250,0.289,0.338,0.271,0.224,0.157,0.132,0.120,0.263,0.261
4,4,0.000,0.000,0.092,0.120,0.120,0.120,0.183,0.042,0.127,...,0.035,0.141,0.042,0.000,0.063,0.000,0.000,0.035,0.106,0.000


In [ ]:

## 브랜드가 0 ~ 3169 이므로 다른 데이터 값에 비해 너무 커서 0 ~ 1사이의 값으로 변환
scaler = MinMaxScaler()

# 데이터를 2D 배열로 변환합니다.
brand_values = train['브랜드'].values.reshape(-1, 1)

# 스케일링을 합니다.
scaler.fit(brand_values)
train['브랜드'] = scaler.transform(brand_values)

# 동일한 변환을 다른 데이터 프레임에 적용합니다.
brand['브랜드'] = scaler.transform(brand['브랜드'].values.reshape(-1, 1))


In [ ]:
brand = brand.fillna(0)

In [ ]:
train.to_csv('train_new.csv', index=False)
brand.to_csv('brand_new.csv', index=False)

# make data array that is train data

In [ ]:
train = pd.read_csv('train_new.csv')
price = pd.read_csv('price_new.csv')
brand = pd.read_csv('brand_new.csv')

print(train.isnull().sum().sum())
print(price.isnull().sum().sum())
print(brand.isnull().sum().sum())

0
0
0


In [ ]:
data_array = []

for i in tqdm(range(train.shape[0])):

    # 시계열 데이터 정보와 가격 정보를 concat (두 데이터는 시리즈)
    df = pd.concat([train.iloc[i, 3:], price.iloc[i, 2:]], axis=1)

    df.columns = ['판매량', '일가격'] # 컬럼명 변경


    # [대분류, 브랜드] 칼럼의 값 (시계열 데이터 전까지의 정보)
    encode_info = dict(train.iloc[i, :3])

    # 시계열 값이 아닌 데이터는 row만큼 데이터를 만든 뒤, 새로운 칼럼으로 만들어서 추가
    for j in encode_info:
        encode_info_ele = [encode_info[j]] * len(df) # row 길이만큼 반복
        df[j] = encode_info_ele


    # 키워드 검색량 칼럼 추가
    df['키워드'] = brand[brand['브랜드'] == df['브랜드'].iloc[0]].iloc[0, 1:]


    # 판매량 * 키워드 칼럼 추가
    df['판매량_키워드'] = df['판매량'] * df['키워드']

    # 인덱스를 시계열로 변경
    df.index = pd.to_datetime(df.index)


    df=df[['대분류',  '브랜드', '키워드', '판매량_키워드', '일가격', '판매량']]

    data_array.append(list(np.array(df)))

data_array = np.array(data_array)
data_array = np.array(data_array, dtype=float)  # 배열을 부동 소수점으로 변환


if np.isnan(data_array).any():
    print("Found NaN values in the data_array!")
else:
    print("No NaN values found in the data_array!")

  0%|          | 0/15890 [00:00<?, ?it/s]

No NaN values found in the data_array!


In [ ]:
train_data = data_array
train_data.shape

(15890, 458, 6)

In [ ]:
def split_data(data, train_len, predict_len):
    train_input = []
    train_target = []

    total_len = train_len + predict_len

    for data_item in tqdm(data):  # for each data item in arr
        for i in range(len(data_item)):
            if i + total_len <= len(data_item):
                train_input.append(data_item[i:i+train_len])
                train_target.append(data_item[i+train_len: i+total_len, -1])

    return np.array(train_input), np.array(train_target)


def split_data2(data, train_len):
    train_input = []

    for data_item in tqdm(data):  # for each data item in arr
        if len(data_item) >= train_len:
            train_input.append(data_item[-train_len:]) # take the last train_len days

    return np.array(train_input)



# split data into train input and target sets
train_len = 90
predict_len = 21
train_input, train_target = split_data(train_data, train_len, predict_len)


# split data into train input and target sets
test_len = 90
test_input = split_data2(train_data, train_len)


# Train / Validation Split
data_len = len(train_input)
val_input = train_input[-int(data_len * 0.2):]
val_target = train_target[-int(data_len * 0.2):]
train_input = train_input[:-int(data_len * 0.2)]
train_target = train_target[:-int(data_len * 0.2)]

  0%|          | 0/15890 [00:00<?, ?it/s]

  0%|          | 0/15890 [00:00<?, ?it/s]

In [ ]:
CFG = {
    'TRAIN_WINDOW_SIZE':60, # 60일치로 학습
    'PREDICT_SIZE':21, # 21일치 예측
    'EPOCHS':10,
    'LEARNING_RATE':2e-4,
    'BATCH_SIZE':4096,
    'SEED':41
}

def seed_everything(seed):
    random.seed(seed)
    os.environ['PYTHONHASHSEED'] = str(seed)
    np.random.seed(seed)
    torch.manual_seed(seed)
    torch.cuda.manual_seed(seed)
    torch.backends.cudnn.deterministic = True
    torch.backends.cudnn.benchmark = True

seed_everything(CFG['SEED']) # Seed 고정

In [ ]:
class CustomDataset(Dataset):
    def __init__(self, X, Y):
        self.X = X.astype(np.float32)  # self.X의 데이터 타입을 float32로 변환
        self.Y = Y.astype(np.float32) if Y is not None else None  # self.Y의 데이터 타입도 float32로 변환

    def __getitem__(self, index):
        if self.Y is not None:
            return torch.from_numpy(self.X[index]), torch.from_numpy(self.Y[index])
        return torch.from_numpy(self.X[index])

    def __len__(self):
        return len(self.X)


train_dataset = CustomDataset(train_input, train_target)
train_loader = DataLoader(train_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=True, num_workers=0)

val_dataset = CustomDataset(val_input, val_target)
val_loader = DataLoader(val_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

In [ ]:
# 필요 없는 변수 제거 (시스템 RAM 많이 절약됨)

del train, price, brand
del data_array
del train_data, train_input, train_target
del val_input, val_target
del train_dataset, val_dataset

# RUN LSTM

In [ ]:
import torch
import torch.nn as nn
import torch.nn.utils as torch_utils

class BaseModel(nn.Module):
    def __init__(self, input_size=6, hidden_size=512, output_size=CFG['PREDICT_SIZE'], dropout_rate=0.2):
        super(BaseModel, self).__init__()
        self.hidden_size = hidden_size

        self.lstm1 = nn.LSTM(input_size, hidden_size, batch_first=True)
        self.bn1 = nn.BatchNorm1d(hidden_size)
        self.lstm2 = nn.LSTM(hidden_size, hidden_size//2, batch_first=True)
        self.bn2 = nn.BatchNorm1d(hidden_size//2)

        self.fc = nn.Sequential(
            nn.Linear(hidden_size//2, hidden_size//4),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.BatchNorm1d(hidden_size//4),
            nn.Linear(hidden_size//4, hidden_size//8),
            nn.ReLU(),
            nn.Dropout(dropout_rate),
            nn.BatchNorm1d(hidden_size//8),
            nn.Linear(hidden_size//8, output_size)
        )

        self.actv = nn.ReLU()

    def forward(self, x):
        batch_size = x.size(0)
        hidden1 = self.init_hidden(batch_size, self.hidden_size, x.device)
        hidden2 = self.init_hidden(batch_size, self.hidden_size//2, x.device)

        lstm_out1, hidden1 = self.lstm1(x, hidden1)
        lstm_out1 = self.bn1(lstm_out1[:, -1, :])
        lstm_out2, hidden2 = self.lstm2(lstm_out1.unsqueeze(1), hidden2)
        lstm_out2 = self.bn2(lstm_out2[:, -1, :])

        output = self.actv(self.fc(lstm_out2))

        return output.squeeze(1)


    def init_hidden(self, batch_size, size, device):
        # Initialize hidden state and cell state
        return (torch.zeros(1, batch_size, size, device=device),
                torch.zeros(1, batch_size, size, device=device))



def train(model, optimizer, train_loader, val_loader, device,clip_value=1.0):
    model.to(device)
    criterion = nn.MSELoss().to(device)
    best_loss = 9999999
    best_model = None

    for epoch in range(1, CFG['EPOCHS'] + 1):
        model.train()
        train_loss = []
        train_mae = []
        for X, Y in tqdm(iter(train_loader)):
            X = X.to(device)
            Y = Y.to(device)

            optimizer.zero_grad()

            output = model(X)
            loss = criterion(output, Y)

            loss.backward()

            torch_utils.clip_grad_norm_(model.parameters(), clip_value)

            optimizer.step()

            train_loss.append(loss.item())

        val_loss = validation(model, val_loader, criterion, device)
        print(f'Epoch : [{epoch}] Train Loss : [{np.mean(train_loss):.5f}] Val Loss : [{val_loss:.5f}]')

        if best_loss > val_loss:
            best_loss = val_loss
            best_model = model
            print('Model Saved')
    return best_model


def validation(model, val_loader, criterion, device):
    model.eval()
    val_loss = []

    with torch.no_grad():
        for X, Y in tqdm(iter(val_loader)):
            X = X.to(device)
            Y = Y.to(device)

            output = model(X)
            loss = criterion(output, Y)

            val_loss.append(loss.item())
    return np.mean(val_loss)

In [ ]:
model = BaseModel()
optimizer = torch.optim.Adam(params = model.parameters(), lr = CFG["LEARNING_RATE"])
infer_model = train(model, optimizer, train_loader, val_loader, device)

  0%|          | 0/1081 [00:00<?, ?it/s]

  0%|          | 0/271 [00:00<?, ?it/s]

Epoch : [1] Train Loss : [0.02363] Val Loss : [0.02027]
Model Saved


  0%|          | 0/1081 [00:00<?, ?it/s]

  0%|          | 0/271 [00:00<?, ?it/s]

...

Epoch : [3] Train Loss : [0.01793] Val Loss : [0.01837]
Model Saved


  0%|          | 0/1081 [00:00<?, ?it/s]

ERROR:root:Internal Python error in the inspect module.
Below is the traceback from this internal error.



Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 3553, in run_code
    exec(code_obj, self.user_global_ns, self.user_ns)
  File "<ipython-input-10-cad46a55200e>", line 3, in <cell line: 3>
    infer_model = train(model, optimizer, train_loader, val_loader, device)
  File "<ipython-input-9-446c7d850685>", line 70, in train
    loss.backward()
  File "/usr/local/lib/python3.10/dist-packages/torch/_tensor.py", line 487, in backward
    torch.autograd.backward(
  File "/usr/local/lib/python3.10/dist-packages/torch/autograd/__init__.py", line 200, in backward
    Variable._execution_engine.run_backward(  # Calls into the C++ engine to run the backward pass
KeyboardInterrupt

During handling of the above exception, another exception occurred:

Traceback (most recent call last):
  File "/usr/local/lib/python3.10/dist-packages/IPython/core/interactiveshell.py", line 2099, in showtraceback
    stb = value._render_tracebac

TypeError: ignored

# Predict

In [ ]:
test_dataset = CustomDataset(test_input, None)
test_loader = DataLoader(test_dataset, batch_size = CFG['BATCH_SIZE'], shuffle=False, num_workers=0)

In [ ]:
def inference(model, test_loader, device):
    predictions = []
    with torch.no_grad():
        for X in tqdm(iter(test_loader)):
            X = X.to(device)
            output = model(X)
            predictions.extend(output.cpu().numpy())  # 이 부분 제거
    return np.array(predictions)


pred = inference(infer_model, test_loader, device)
print(pred)

  0%|          | 0/4 [00:00<?, ?it/s]

[[0.         0.         0.00187076 ... 0.04272572 0.04407267 0.04475202]
 [0.06505691 0.07396591 0.08116855 ... 0.08398206 0.08646232 0.08930076]
 [0.         0.         0.         ... 0.02240173 0.0243873  0.02706562]
 ...
 [0.         0.         0.         ... 0.         0.         0.        ]
 [0.08284494 0.08427466 0.0897523  ... 0.08997352 0.08907405 0.08731652]
 [0.         0.         0.         ... 0.         0.         0.        ]]


In [ ]:
import pickle

with open("scale_min_dict.pkl","rb") as fr:
  scale_min_dict = pickle.load(fr)
with open("scale_max_dict.pkl","rb") as fr:
  scale_max_dict=pickle.load(fr)


for idx in range(len(pred)):
  pred[idx, :] = pred[idx, :] * (scale_max_dict[idx] - scale_min_dict[idx]) + scale_min_dict[idx]

# 결과 후처리
pred = np.round(pred)
pred = pred.astype(int)

In [ ]:
submit = pd.read_csv('sample_submission.csv')

submit.iloc[:,1:] = pred
submit.head()

,ID,2023-04-05,2023-04-06,2023-04-07,2023-04-08,2023-04-09,2023-04-10,2023-04-11,2023-04-12,2023-04-13,...,2023-04-16,2023-04-17,2023-04-18,2023-04-19,2023-04-20,2023-04-21,2023-04-22,2023-04-23,2023-04-24,2023-04-25
0,0,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,1,1,1,1
1,1,1,1,1,1,1,1,1,1,1,...,1,1,1,1,1,1,1,1,1,1
2,2,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,1,1,1,1,1
3,3,0,0,0,0,0,0,0,0,0,...,0,0,0,1,1,1,2,2,2,2
4,4,0,0,0,0,0,0,0,0,0,...,0,0,0,0,0,0,0,0,0,1


In [ ]:
submit.to_csv('submit_final.csv', index=False)